# Optimizing Your Scripts for the Cloud and TIKE

---

When running scripts that process large amounts of information, optimizing your code is essential to reducing computational costs and improving efficiency. In this tutorial, we will explore tools and techniques for optimizing your scripts on the [TESS Integrated Knowledge Engine (TIKE)](https://timeseries.science.stsci.edu/). 

## Table of Contents

1. [Imports and Setup](#imports-and-setup)
2. [TESS Image Cutouts](#tess-image-cutouts)
3. [Accessing Cloud Resources](#accessing-cloud-resources)
4. [Profiling Your Scripts](#profiling-your-scripts)
5. [Parallel Processing and Dask](#parallel-processing-and-dask)

## Imports and Setup

We will import the following packages:
- `astropy` to handle coordinates and FITS file handling
- `astroquery.mast` to search for and select data
- `s3fs` to access cloud-hosted data
- `lightkurve` to analyze astronomical time series data
- `dask` to parallelize workflows

In [1]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astroquery.mast import Observations, Tesscut
import dask.array as da
import lightkurve as lk
import numpy as np
import s3fs
%matplotlib inline

We will also enable cloud data access in `astroquery.mast`. This will allow us to fetch the cloud URIs for data products and access files directly without downloading them.

In [ ]:
Observations.enable_cloud_dataset()

## TESS Image Cutouts

[TESS full-frame images (FFIs)](https://heasarc.gsfc.nasa.gov/docs/tess/Full-Frame-Image-Tutorial.html) are very large, and processing or transferring them can be computationally expensive. Making cutouts of these images significantly reduce data size, making scripts faster to process and easier to handle in terms of memory and storage. They focus analysis on specific targets, eliminating the need to crop or filter unnecessary regions.

### Requesting an FFI Cutout

[TESSCut](https://mast.stsci.edu/tesscut/) is MAST's tool to provide cutouts of TESS FFIs. Cutouts can be made from either the Science Processing Operation's Center (SPOC) FFI products, or the [TESS Image CAlibrator (TICA)](https://archive.stsci.edu/hlsp/tica) high-level science products. The cutouts are returned in the form of [target pixel files](https://heasarc.gsfc.nasa.gov/docs/tess/Target-Pixel-File-Tutorial.html) in the same format as TESS pipeline target pixel files. This tool can be accessed with `astroquery.mast` by using the [`Tesscut`](https://astroquery.readthedocs.io/en/latest/mast/mast_cut.html#tesscut) class.

### Getting Sector Information

The TESS mission has surveyed more than [93% of the entire sky](https://www.nasa.gov/universe/nasas-tess-celebrates-fifth-year-scanning-the-sky-for-new-worlds/). To return the sectors that contain a particular coordinate, object, or moving target, you can use the `get_sectors` function.

In [ ]:
# Get the sectors that contain a coordinate
coord = SkyCoord(135.1408, -5.1915, unit='deg')
sector_table = Tesscut.get_sectors(coordinates=coord)
sector_table  

In [ ]:
# Get the sectors that contain a moving target
mt_table = Tesscut.get_sectors(objectname='Ceres', moving_target=True)
mt_table

### Getting Cutouts

The `get_cutouts` method creates a cutout target pixel file of an FFI according to the following parameters:

- `coordinates`: Coordinates around which to search.
- `objectname`: Object around which to search.
- `size`: Size of cutout array.
- `product`: Product from which cutouts are made ('SPOC', 'TICA').
- `sector`: The TESS sector to return the cutout from. If not supplied, cutouts from all available sectors in which the target appears will be returned.
- `moving_target`: Whether an object is a moving target or not.

The returned object is a list of `~astropy.io.fits.HDUList` objects, one for each cutout.

In [ ]:
# Make a 10x10 cutout around target coordinates
hdulist = Tesscut.get_cutouts(coordinates=coord,
                              size=10,
                              product='tica',
                              sector=34)
hdulist[0].info()

In [ ]:
# Make a 15x15 cutout around a moving target
hdulist = Tesscut.get_cutouts(objectname='Ceres',
                              size=15,
                              moving_target=True,
                              sector=29)
hdulist[0].info()

To better visualize a cutout of a moving target, we'll use the `~lightkurve.TessTargetPixelFile.animate` function to create an animation of the TPF we just created. Notice how the background around the target ([Ceres](https://science.nasa.gov/dwarf-planets/ceres/facts/)) changes with each frame as the dwarf planet moves relative to other objects in the sky.

In [ ]:
# Create a TESSTargetPixelFile object using the HDUList object
tpf = lk.TessTargetPixelFile(hdulist[0])

# Animate the TPF
tpf.animate()

## Accessing Cloud Resources

Throughout this workshop, we've talked extensively about the benefits of working on a cloud platform like the TIKE. One important plus is that working on the cloud gives you direct, in-memory access to cloud-hosted data products without having to download them to your local machine. This speeds up data access and allows you to perform analyses that require large amounts of data without overloading your local storage.

In this section, we'll explore a few tools and methods for accessing TESS data products on the cloud. All of these tools load data directly into memory, and some even allow you to perform further analyses and visualizations. 

To start off, we will use the `astroquery.mast` module to perform a query around a TESS target. We will then use the `Observations.get_cloud_uris` function to get the cloud URIs for light curve products and target pixel file products that are associated with observations returned by the query criteria. You may recognize this workflow from the earlier session about *Querying for TESS Data in MAST*!

In [8]:
# Define query criteria for a certain TESS target. Only return timeseries data products from sectors 15, 16, and 17
query_criteria = {'target_name': 375422201,
                  'obs_collection': 'TESS',
                  'dataproduct_type': 'timeseries',
                  'sequence_number': [15, 16, 17]}

# Get cloud URIs for light curve products
lc_uris = Observations.get_cloud_uris(**query_criteria,
                                      filter_products={'productType': 'SCIENCE',
                                                       'productSubGroupDescription': 'LC'})

# Get cloud URIs for target pixel file products
tpf_uris = Observations.get_cloud_uris(**query_criteria,
                                       filter_products={'productType': 'SCIENCE',
                                                        'productSubGroupDescription': 'TP'})

Now that we have two lists of cloud URIs (`lc_uris` and `tpf_uris`), we can access these data products on the TIKE.

### Opening a Cloud FITS File with Astropy

The [`astropy.io.fits`](https://docs.astropy.org/en/stable/io/fits/index.html) module allows you to access data in [Flexible Image Transport System (FITS)](https://heasarc.gsfc.nasa.gov/docs/heasarc/fits_overview.html) files. FITS is a portable file standard widely used in astronomy to store images and tables. The [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/) package is an optional dependency of `astropy` that supports file reading from a range of remote and distributed storage backends, like [Amazon S3](https://aws.amazon.com/s3/). [STScI has a registry of open data](https://registry.opendata.aws/collab/stsci/) on AWS that includes data from TESS.

The `fits.open` function accepts two parameters related to cloud file access:
- `use_fsspec`: Whether to use the `fsspec.open` method to open the file. Essentially, whether or not the file is a cloud file.
- `fsspec_kwargs`: Keyword arguments passed to `fsspec.open`. This can be used to configure cloud storage credentials and caching behavior. For example, pass `fsspec_kwargs={"anon": True}` to enable anonymous access to Amazon S3 open data buckets. If this parameter is defined, then `use_fsspec` is assumed to be `True`.

In [ ]:
# Access cloud FITS file anonymously and print its info
with fits.open(lc_uris[0], fsspec_kwargs={'anon': True}) as hdu:
    hdu.info()

### Opening a Cloud File with s3fs

You may wish to open a cloud file that is not in FITS format, and in this case, the [`s3fs`](https://s3fs.readthedocs.io/en/latest/#) package will come in handy. `s3fs` is a Pythonic file interface to Amazon S3 that allows you to browse and access cloud files as if they were local. 

You can access data from cloud files using the `s3fs.S3FileSystem.open` function and perform further operations within the function's context. Below, we open a FITS file with `s3fs` and open it again with `astropy.io.fits` to access its content. If we were accessing another filetype like XML or PDF, we would want to use other libraries/functions that are built to handle them.

In [ ]:
# Initialize the S3 filesystem for an anonymous user
fs = s3fs.S3FileSystem(anon=True)

# Open the file with s3fs
with fs.open(lc_uris[0]) as f:
    # Open the file's contents with astropy.io.fits
    with fits.open(f) as hdu:
        hdu.info()

### Reading Cloud Files with Lightkurve

[`lightkurve`](https://lightkurve.github.io/lightkurve/#) is an open-source Python package that offers user-friendly ways to analyze astronomical time series data. Recent updates to `lightkurve` allow users to read data products from the cloud directly into memory. These data products must be in FITS format.

To read a single data product, simply use the `lightkurve.io.read` method and pass in a cloud URI for either a light curve file or a target pixel file. The function will determine the type of the file and return the corresponding object. From here, you have access to a plethora of attributes and methods for analyzing and visualizing the data.

For a [`LightCurve` object](https://lightkurve.github.io/lightkurve/reference/lightcurve.html), these include, but are not limited to:
- `LightCurve.time`: Time values stored as an `astropy.time.Time` object.
- `LightCurve.flux`: Brightness values stored as an `astropy.units.Quantity` object.
- `LightCurve.plot()`: Plot the light curve.
- `LightCurve.fold()`: Fold the light curve on a given period and epoch.
- `LightCurve.create_transit_mask()`: Returns a boolean array that is `True` during transits and `False` elsewhere.

Below, we will read and plot a single light curve file from the cloud.

In [ ]:
# Read a single light curve file
lc = lk.io.read(lc_uris[0])
print('Type:', type(lc))

# Plot the light curve
lc.plot()

To read a collection of light curve products from the cloud, we can use the `lightkurve.io.read_lc_collection` method. This returns a [`lightkurve.LightCurveCollection` object](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurveCollection.html?highlight=lightcurvecollection#lightkurve.LightCurveCollection), which holds a collection of `LightCurve` objects and has some additional attributes and methods.

In [ ]:
# Read a collection of light curves
collection = lk.io.read_lc_collection(lc_uris)
print('Type:', type(collection))

# Plot the collection
collection.plot()

Notice how each of the three light curves is plotted in a different color according to the chart legend. To stitch all of the light curves in the collection into a single `LightCurve` object, we can set the `stitch` argument to be `True`. Each light curve will be normalized prior to stitching.

In [ ]:
# Read a collection of light curves as a single, stitched light curve
stitched = lk.io.read_lc_collection(lc_uris, stitch=True)
print('Type:', type(stitched))

# Plot the light curve
stitched.plot()

As mentioned above, we can also read in a target pixel file from the cloud using `lightkurve.io.read`. Simply pass in a cloud URI for a target pixel file.

For a [`TessTargetPixelFile` object](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.TessTargetPixelFile.html#lightkurve.TessTargetPixelFile), available attrubutes and methods include, but are not limited to:
- `TessTargetPixelFile.time`: Time values for all good-quality cadences stored as an `astropy.time.Time` object.
- `TessTargetPixelFile.flux`: Flux values for all good-quality cadences stored as an `astropy.units.Quantity` object.
- `TessTargetPixelFile.shape`: Cube dimension shape.
- `TessTargetPixelFile.plot()`: Plot the pixel data for a given frame.
- `TessTargetPixelFile.animate()`: Displays an interactive animation of the plots for each frame.
- `TessTargetPixelFile.create_threshold_mask()`: Returns an aperture mask created using a thresholding method.

Below, we will read and create an animation for a target pixel file from the cloud.

In [ ]:
# Read in a target pixel file from the cloud
tpf = lk.io.read(tpf_uris[0])
print('Type:', type(tpf))

# Create animation
tpf.animate()

## Profiling Your Scripts

When running scripts that process large amounts of information, optimizing your code is essential to reducing to computational costs and improving efficiency. Profiling tools can help identify bottlenecks in your code, enabling targeted improvements. One such tool is the `line_profiler` Jupyter extension, which provides detailed insights into the execution of individual lines of Python code.

In this section, we’ll explore how to use [`line_profiler`](https://kernprof.readthedocs.io/en/latest/) to analyze and optimize scripts. By pinpointing inefficiencies, you can fine-tune your code to minimize execution time, making it faster, more cost-effective, and better suited for scalable cloud environments. This approach is particularly valuable for computationally intensive tasks, such as processing large datasets or executing iterative algorithms.

`line_profiler` is not automatically installed on the TIKE, so we'll need to use some [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). Magic commands are shell-style commands that can be run in a notebook. They are preceded by a `%` character. We will use magic commands to install the extension with `pip` and then load the extension. You may have to restart your kernel to fully load the extension.

In [ ]:
# Install the line_profiler module
%pip install line_profiler

In [16]:
# Load the extension into the kernel
%load_ext line_profiler

To illustrate the capabilities of `line_profiler`, we'll define a short analysis workflow that you might use with TESS data. The `calculate_flux` function calculates the mean flux for a target across multiple images using an aperture. As input, it accepts an array of images, a pixel position for a target, and an aperture radius. It returns the mean flux of the aperture across all images.

As an example, we'll generate synthetic image data for 100 images of size 2048 x 2048 pixels. We will then calculate the mean flux at position (512, 512) with an aperture radius of 5 pixels.

In [17]:
def generate_image_data(num_images, image_size):
    """Generate synthetic image data simulating flux values."""
    return np.random.random((num_images, image_size, image_size))

# Generate synthetic data for 100 images of size 2048x2048
image_data = generate_image_data(100, 2048)

In [ ]:
def calculate_flux(image_data, x, y, aperture_radius):
    """
    Calculates the mean flux for a target at (x, y) using an aperture.
    
    Parameters:
    - image_data: 3D NumPy array (num_images, image_width, image_height)
    - x, y: Coordinates of the target star.
    - aperture_radius: Radius of the circular aperture.
    
    Returns:
    - Mean flux across all images.
    """
    fluxes = []

    for image in image_data:
        # Create a grid of distances from the target
        y_indices, x_indices = np.ogrid[:image.shape[0], :image.shape[1]]
        distances = np.sqrt((x_indices - x)**2 + (y_indices - y)**2)
        
        # Extract pixels within the aperture radius
        aperture_mask = distances <= aperture_radius

        # Flux is the sum of pixel values in aperture
        aperture_flux = image[aperture_mask].sum()
        fluxes.append(aperture_flux)
    
    return np.mean(fluxes)

# Example usage
mean_flux = calculate_flux(image_data, x=512, y=512, aperture_radius=5)
print(f"Mean flux: {mean_flux}")

In [ ]:
def calculate_flux(image_data, x, y, aperture_radius):
    """
    Calculates the mean flux for a target at (x, y) using an aperture.
    
    Parameters:
    - image_data: 3D NumPy array (num_images, image_width, image_height)
    - x, y: Coordinates of the target star.
    - aperture_radius: Radius of the circular aperture.
    
    Returns:
    - Mean flux across all images.
    """

    # Create a grid of distances from the target
    y_indices, x_indices = np.ogrid[:image_data.shape[1], :image_data.shape[2]]
    distances = np.sqrt((x_indices - x)**2 + (y_indices - y)**2)

    # Extract pixels within the aperture radius
    aperture_mask = distances <= aperture_radius

    # Calculate the flux for each image using the precomputed aperture mask
    fluxes = image_data[:, aperture_mask].sum(axis=1)

    # Return the mean flux
    return np.mean(fluxes)

# Example usage
mean_flux = calculate_flux(image_data, x=512, y=512, aperture_radius=5)
print(f"Mean flux: {mean_flux}")

This example doesn't take more than a few seconds. However, what if you wanted to process a thousand images? A million? This analysis can quickly become unwieldly as its input scales up. `line_profiler` can help us to pinpoint the bottlenecks in this code so that we might make targeted improvements. 

The `%lprun` magic command allows you to specify a function to profile and a statement to execute. The usage is `%lprun -f func <statement>`.

In [ ]:
%lprun -f calculate_flux calculate_flux(image_data, x=512, y=512, aperture_radius=5)

`line_profiler` outputs a table analyzing each line in the `calculate_flux` function. 

- "Hits" is the number of times that the line was executed.
- "Time" is the total amount of time spent executing the line in timer's units.
- "Per Hit" is the average amount of time spent executing the line once in the timer's units.
- "% Time" is the percentage of time spent on that line relative to the total amount of recorded time spent in the function.

There is definitely some room for improvement in this function. Let's implement the following and see how the profiler results change:

1. The majority of the function's time is spent calculating the distances from each pixel in each image to the target position. The second highest execution time is associated with the line creating the aperture mask. Because each image in the array has the same width and height, these calculations are redundant. One way to lessen the effect of this bottleneck is to move these lines out of the for-loop so that they are only executed once.
2. Instead of iterating through every image to calculate its aperture flux, we can apply the aperture mask to all images with slicing (`image_data[:, aperture_mask]`). This takes advantage of NumPy’s vectorized operations, which are more efficient than Python loops.
3. The `sum(axis=1)` array operation can be used to compute the total aperture flux for all images in a single step.

## Parallel Processing and Dask

Parallelization involves splitting a task into smaller sub-tasks that can run simultaneously on multiple processors or cores. Cloud platforms like the TIKE often provide scalable resources (e.g., CPUs, GPUs, distributed nodes), making parallelization an effective way to optimize performance. The TIKE currently has four cores for multi-processing.

In today's tutorial, we'll dive into [`dask`](https://www.dask.org/), a flexible, open-source library for parallel and distributed computing of larger-than-memory datasets. Dask parallelizes libraries like [`numpy`](https://numpy.org/), [`pandas`](https://pandas.pydata.org/), and [`scikit-learn`](https://scikit-learn.org/stable/) and allows them to scale, either on a single machine with multiple cores or on a large distributed cluster. The package allows you to easily transition from traditional, single-machine workflows to multi-core computing without having to learn a new framework or rewrite all of your code.

### Dask Collections

Dask's APIs can be thought of as high-level collections and low-level collections.

<div>
<img src="dask_collections.png" width="500"/>
</div>

[Image Source: Dask Tutorial](https://github.com/dask/dask-tutorial/blob/main/00_overview.ipynb)

The high-level `Dataframe`, `Array`, and `Bag` collections mimic `pandas`, `numpy`, and lists, respectively. They can operate in parallel on datasets that don't fit into memory.

The low-level `Delayed` and `Futures` collections provide finer control to build custom parallel and distributed computations.

### Dask Cluster

Dask uses a distributed scheduler that exists in the context of a Dask cluster, the structure of which is shown below.

<div>
<img src="dask_cluster.png" width="500"/>
</div>

[Image Source: Dask Tutorial](https://github.com/dask/dask-tutorial/blob/main/00_overview.ipynb)

### `Array` Example

Dask is a large library with many features, and we could probably spend an entire session on this package alone. In the interest of time and to stay within the scope of this tutorial, we will walk though a single, TESS-relevant example that can be optimized with Dask's `Array` collection.

In the following cell, we define a function to generate random data for a certain number of images of a certain size. We then call this function to create sample image data for 300 images that are 2048 x 2048 pixels. Finally, we calculate and print the mean of this random data.


In [ ]:
%%time
def generate_image_data(num_images, image_size):
    """Generate synthetic image data simulating flux values."""
    return np.random.random((num_images, image_size, image_size))

# Create 300 images of size 2048 x 2048
image_data = generate_image_data(300, 2048)

# Compute the mean of the array
mean = image_data.mean()

print(f"Mean: {mean}")

The execution time of this cell will vary, but it usually takes around 12 seconds on the TIKE.

Now, let's modify this code to use Dask's `Array` collection. We should see a significant improvement in performance.

In [ ]:
%%time
def generate_image_data_dask(num_images, image_size):
    """Generate synthetic image data simulating flux values using Dask."""
    return da.random.random((num_images, image_size, image_size), chunks=(10, 1000, 1000))

# Create 1000 images of size 2048 x 2048
image_data = generate_image_data_dask(300, 2048)

# Compute the mean of the array
mean = image_data.mean().compute()

print(f"Mean: {mean}")

### Distributed Cloud Computing, Coming Soon!

[Dask Gateway](https://gateway.dask.org/) is a tool that deploys and manages Dask clusters on shared infrastructure like a cloud platform. It allows users to launch and manage their own clusters with secure and scalable configurations. In the future, we hope to integrate Dask Gateway into the TIKE so that users have access to the combined power and resources of multiple machines. 

<div>
<img src="dask_gateway.png" width="500"/>
</div>

[Image Source: Dask Gateway Docs](https://gateway.dask.org/)